Tanner, Noah\
HW 0x03 Systems Modeling\
ME 405 - Professor Refvem\
Fall 2023

HW 0x03 Systems Modeling
ME 405 - Professor Refvem
Fall 2023

In [2]:
from numpy import array, arange, zeros, transpose
from matplotlib import pyplot, rc
from math import pi

Systems Model:\
\
In order to model the Romi robot system as a differential drive robot, and not a "unicycle" model, the independent characteristics of each wheel must be considered. In the image below, there is an independent velocity vector for the left and right wheels, as well as a combined velocity vector that makes up the velocity at a central point, inbetween the two wheels. The two coordinate systems, X,Y (Global), and X_R, Y_R (Local to the robot) are also pictured.
\
<img src="./Model_Pic.png" alt="Diagram of Romi Model" style="height:300px;" />
\
Some variables have been defined as well in order to understand the equations.
\
\
R_r     = Radius of left wheel\
R_l     = Radius of right wheel\
R       = Radius (R_r = R-l = R, unless otherwise noted)\
L       = Distance between the two wheels (Radius of rotation)

The following differential equations reflect the dynamics associated with a two wheel, differential drive robot. 

$$
\begin{array}{rl}
\frac{d}{dt}X           &= \left( \frac{R}{2} \left( \omega_r + \omega_l \right) 
\cos\theta \right)\\
\frac{d}{dt}Y           &= \left( \frac{R}{2} \left( \omega_r + \omega_l \right) 
\sin\theta \right)\\
\frac{d}{dt}\theta      &= \left( \frac{R}{L} \left( \omega_r + \omega_l \right) 
\right)
\end{array}
$$

Combining the preceding equations, a single vector equation can be obtained that can be solved using vector based ODE solution techniques.

$$
\begin{array}{rl}
\frac{d}{dt}
    \begin{bmatrix}
        X \\
        Y \\
        \theta
    \end{bmatrix}
&=
\begin{bmatrix}
    \frac{R}{2} \left( \omega_r - \omega_l \right)cos \theta \\
    \frac{R}{2} \left( \omega_r - \omega_l \right)sin \theta \\
    \frac{R}{L} \left( \omega_r - \omega_l \right)
\end{bmatrix} \\

\frac{d}{dt}
    \begin{bmatrix}
        X \\
        Y \\
        \theta
    \end{bmatrix}
&=
\begin{bmatrix}
    0 & 0 & 0 \\
    0 & 0 & 0 \\
    0 & 0 & 0 \\
\end{bmatrix}
    \begin{bmatrix}
        X \\
        Y \\
        \theta
    \end{bmatrix}
+
\begin{bmatrix}
    \frac{R}{L} cos\theta & \frac{R}{L} cos\theta \\
    \frac{R}{L} sin\theta & \frac{R}{L} sin\theta \\
    \frac{R}{L} & \frac{R}{L}
\end{bmatrix}
    \begin{bmatrix}
        \omega_r \\
        \omega_L
    \end{bmatrix}
\end{array}
$$

